In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')



In [2]:
try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

### Get Data

In [3]:
games = pd.read_csv(DATAPATH / "games.csv")

### Data Processing

In [4]:
games = process_games(games) 
games = add_TARGET(games)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Feature Engineering

In [5]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

games = process_features(games)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HOME_TEAM_ID
['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'TARGET', 'MONTH', 'HOME_TEAM_WIN_STREAK', 'HOME_TEAM_WINS_AVG_LAST_3_HOME', 'HOME_TEAM_WINS_AVG_LAST_7_HOME', 'HOME_TEAM_WINS_AVG_LAST_10_HOME', 'HOME_PTS_home_AVG_LAST_3_HOME', 'HOME_PTS_home_AVG_LAST_7_HOME', 'HOME_PTS_home_AVG_LAST_10_HOME', 'HOME_FG_PCT_home_AVG_LAST_3_HOME', 'HOME_FG_PCT_home_AVG_LAST_7_HOME', 'HOME_FG_PCT_home_AVG_LAST_10_HOME', 'HOME_FT_PCT_home_AVG_LAST_3_HOME', 'HOME_FT_PCT_home_AVG_LAST_7_HOME', 'HOME_FT_PCT_home_AVG_LAST_10_HOME', 'HOME_FG3_PCT_home_AVG_LAST_3_HOME', 'HOME_FG3_PCT_home_AVG_LAST_7_HOME', 'HOME_FG3_PCT_home_AVG_LAST_10_HOME', 'HOME_AST_home_AVG_LAST_3_HOME', 'HOME_AST_home_AVG_LAST_7_HOME', 'HOME_AST_home_AVG_LAST_10_HOME', 'HOME_REB_home_AVG_LAST_3_HOME', 'HOME_REB_home_AVG_LAS

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['GAME_DATE_EST', 'GAME_ID', 'TEAM1', 'TEAM1_home', 'TEAM2', 'TEAM1_win', 'SEASON', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'MATCHUP_WINPCT_3', 'MATCHUP_WINPCT_7', 'MATCHUP_WINPCT_10', 'MATCHUP_WIN_STREAK', 'WIN_STREAK', 'HOME_AWAY_STREAK', 'TEAM1_win_AVG_LAST_3_ALL', 'TEAM1_win_AVG_LAST_7_ALL', 'TEAM1_win_AVG_LAST_10_ALL', 'TEAM1_win_AVG_LAST_15_ALL', 'PTS_AVG_LAST_3_ALL', 'PTS_AVG_LAST_7_ALL', 'PTS_AVG_LAST_10_ALL', 'PTS_AVG_LAST_15_ALL', 'FG_PCT_AVG_LAST_3_ALL', 'FG_PCT_AVG_LAST_7_ALL', 'FG_PCT_AVG_LAST_10_ALL', 'FG_PCT_AVG_LAST_15_ALL', 'FT_PCT_AVG_LAST_3_ALL', 'FT_PCT_AVG_LAST_7_ALL', 'FT_PCT_AVG_LAST_10_ALL', 'FT_PCT_AVG_LAST_15_ALL', 'FG3_PCT_AVG_LAST_3_ALL', 'FG3_PCT_AVG_LAST_7_ALL', 'FG3_PCT_AVG_LAST_10_ALL', 'FG3_PCT_AVG_LAST_15_ALL', 'AST_AVG_LAST_3_ALL', 'AST_AVG_LAST_7_ALL', 'AST_AVG_LAST_10_ALL', 'AST_AVG_LAST_15_ALL', 'REB_AVG_LAST_3_ALL', 'REB_AVG_LAST_7_ALL', 'REB_AVG_LAST_10_ALL', 'REB_AVG_LAST_15_ALL']
['PTS_AVG_LAST_3_ALL', 'PTS_AVG_LAST_7_ALL', 'PTS_AVG

### Backfill Feature Store

In [6]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Rolling Stats Features**

In [7]:
rolling_stats_fg = fs.get_or_create_feature_group(
    name="rolling_stats",
    version=1,
    description="Rolling averages and current win/lose streaks",
    #primary_key=["GAME_ID"],
    primary_key = ['GAME_DATE_EST','HOME_TEAM_ID'],
    event_time="game_date_est", #must be lowercase
)

In [8]:
rolling_stats_fg.insert(games, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/22547 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x25f75cd4220>, None)